<a href="https://colab.research.google.com/github/brunodleite/alura_machine_learning_validacao_modelos/blob/main/validacao_cruzada_aleatoriedade_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


Separando os dados de treino e teste para realizar a comparação com outros modelos

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25,
                                                        stratify=y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


Antes de rodar o classificador, precisamos decidir um taxa de acerto aceitável.

Para se ter como base será utilizado um DUMMY CLASSIFIER, que é utilizado para se ter um fator de comparação com outros classificadores complexos. No DUMMY CLASSIFIER é realizado uma classificação através de regras simples. É de se esperar que qualquer outro classificador complexo terá de ser melhor que o DUMMY CLASSIFIER.

Estratégias utilizadas pelo DUMMY CLASSIFIER:


1.   Mais frequente: o classificador sempre prevê o rótulo a classe mais frequente nos dados de treinamento
2.   Estratificado: Gerar previsões respeitando a distribuição das classes dos dados de treino.
3.   Uniforme: Gera previsões uniformemente de forma aleatória.
4.   Constante: O classificador sempre prevê um rótulo constante e é usado principalmente ao classificar rótulos de classe não majoritária


A Taxa de acerto simples foi 58%

In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A Acurácia do dummy stratified foi %.2f%%" % acuracia)

A Acurácia do dummy stratified foi 58.00%


Rodando a DecisionTreeClassifier 

O problema é que cada vez que rodamos o algoritmo com um seed diferente, resulta em acuracidades diferentes. Um problema chamado de holdout, sendo arriscado para aplicações em modelos baseados na vida real.

In [7]:
from sklearn.tree import DecisionTreeClassifier 

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 71.92%


Exemplo alterando o SEED

In [8]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25,
                                                       stratify=y)
#print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acuracia foi %.2f%%" % acuracia)

A acuracia foi 76.84%


Aplicando o cross_validate e testando as acuracidades com diferentes CV (Parâmetro que determina a quantidade de cortes para treino e teste), utilizando intervalos de confianças para deixar o modelos mais confiável.

In [9]:
from sklearn.model_selection import cross_validate
SEED = 12
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation, 3 = [74.99, 76.57]


O parâmetro utilizado será **cv = 5**, conforme o indicado pela documentação do sklearn.

[Doc.CrossValidation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

In [10]:
from sklearn.model_selection import cross_validate
SEED = 12
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation, 5 = [75.21, 76.35]
